In [1]:
import os

from IPython.display import display,clear_output
from ultralytics import YOLO
import time
import chess.svg
from IPython.display import SVG
import chess
from helper_functions import *
import warnings
warnings.filterwarnings("ignore")

from stockfish_downloader import download_stockfish
stockfish_path = download_stockfish()

Detected OS: Darwin, Architecture: arm64
Stockfish is already downloaded. path: /Users/akhil838/Documents/PyCharm_Projects/MiniProject/stockfish/stockfish-macos-m1-apple-silicon


In [2]:
# model = YOLO('./runs/detect/train22/weights/best.pt')
model = YOLO('model/best.pt')

In [3]:
cameras = get_available_cameras()
if cameras:
    print("Available Cameras:", cameras)
else:
    print("No cameras found.")

Available Cameras: [0, 1]


OpenCV: out device of bound (0-1): 2
OpenCV: camera failed to properly initialize!
OpenCV: out device of bound (0-1): 3
OpenCV: camera failed to properly initialize!
OpenCV: out device of bound (0-1): 4
OpenCV: camera failed to properly initialize!


In [5]:
engine = start_stockfish(stockfish_path)
print(stockfish_nextmove(engine, '8/7k/4r3/8/7K/8/8/7P'))

Stockfish Initialized
e6g6


In [ ]:
import cv2
dev_mode = False
# Load YOLO model (ensure you have the correct model path)
# Open webcam (0 for default, 1 for external camera)
cap = cv2.VideoCapture(1)
engine = start_stockfish('./stockfish/stockfish-macos-m1-apple-silicon')
frame_count = 0
prev_fen = ''
next_move = None
while cap.isOpened():

    start_time = time.time()
    ret, frame = cap.read()  # Read frame from webcam
    if not ret:
        break
    frame_count += 1

    # Run YOLO inference
    results = model.predict(frame, conf=0.3, device="mps", verbose=False)  # Adjust confidence threshold as needed

    board_corners = []
    board_bbox = []

    pieces = []
    for result in results:
        if result.boxes is None or len(result.boxes) == 0:
            continue  # Skip frame if no detections

        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())  # Convert tensor to integer numpy array

            # Draw bounding box


            # Draw label
            conf = box.conf[0].item()
            cls = int(box.cls[0].item())
            label = f"{model.names[cls]} {conf:.2f}"
            if 'board_corner' == model.names[cls]:
                board_corners.append(((x1+x2)/2, (y1+ y2)/2))
                board_bbox.append((x1, y1, x2, y2))

                cv2.drawMarker(frame, ((x1 + x2)// 2, (y1 + y2) //2), (0, 0, 255), 1,18,6)
                #cv2.putText(frame, label, (x1, y1 - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            else:
                pieces.append(((x1,y1), (x2,y2), label))

    intersection_points = []
    if len(board_bbox) == 4 and not check_any_bbox_overlap(board_bbox):
        frame, intersection_points = draw_perspective_grid(frame, board_corners,grid_size=8,dev_mode=dev_mode)
        if dev_mode:
            for row in intersection_points:
                #print(intersection_points)
                for i,j in row:
                    cv2.drawMarker(frame, (i, j), (0, 255, 255), 2)

    perspective_pieces = []
    for (x1,y1),(x2,y2),label in pieces:
        trans_piece = list(map(int,get_perspective_piece((x1,y1,x2,y2),board_corners)))
        perspective_pieces.append([trans_piece,label.split()[0][:-1]])
        if dev_mode:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            cv2.drawMarker(frame,list(trans_piece),(255, 255, 255), markerType=2, markerSize=15, thickness=1)

    if intersection_points:
        board = get_board(intersection_points, perspective_pieces)
        #print(board)
        if board:
            FEN = ''
            for row in board:
                empty = 0
                #print(row)
                for col in row+' ':
                    #print(col)
                    if col != '.':
                        if empty != 0:
                            FEN += str(empty)
                            empty = 0
                        if col != ' ':
                            FEN += col
                    else:
                        empty += 1

                FEN += '/'
            FEN = FEN[:-1]

            clear_output()
            board = chess.Board(FEN)
            custom_colors = {
                "square light": "#eeeed2",  # White squares
                "square dark": "#769656",   # Green squares
            }
            # Generate the SVG with custom colors
            svg_board = chess.svg.board(board=board, colors=custom_colors,size = 350)

            display(SVG(svg_board),frame_count)

            display_chess_fen(FEN)
            if prev_fen != FEN:
                prev_fen = FEN
                next_move = stockfish_nextmove(engine, FEN)

            if next_move is None:
                engine = start_stockfish('./stockfish/stockfish-macos-m1-apple-silicon')
            else:
                frame = augument_nextMove(frame, next_move,intersection_points)

            cv2.putText(frame, next_move,(100,100),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)

    #print(perspective_pieces)

    #print(frame.shape)
    frame = cv2.resize(frame, (2560,1440))
    # Show frame
    cv2.imshow("YOLO Detection", frame)
    end_time = time.time()
    display(f'Inference Time: {(end_time-start_time)*100:.4f} ms')
    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
engine.stdin.write('quit\n')
engine.stdin.flush()
engine.terminate()

Stockfish Initialized


'Inference Time: 133.8298 ms'

'Inference Time: 3.2795 ms'

'Inference Time: 4.0276 ms'

'Inference Time: 2.6230 ms'

...

'Inference Time: 3.2721 ms'

'Inference Time: 2.4071 ms'

'Inference Time: 3.2246 ms'

'Inference Time: 2.1255 ms'

'Inference Time: 3.2315 ms'